In [ ]:
# %pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1

ERROR: Could not find a version that satisfies the requirement torch==2.0.0 (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==2.0.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install torch==2.0.0 torchdata

  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchdata-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_nccl_cu11-2.14.3-py3-none-manylinux1_x86_64.whl (177.1 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_

In [5]:
%pip install --disable-pip-version-check -q \
    transformers==4.27.2 \
    datasets==2.9.0 \
    accelerate==0.17.0 \
#    promptsource==0.2.3 \
    evaluate==0.4.0 \
    trl==0.4.1
    # rouge_score==0.1.2 \
    # loralib==0.1.1

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install git+https://github.com/lvwerra/trl.git

  Cloning https://github.com/lvwerra/trl.git to /tmp/pip-req-build-l1fxp5k_
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-l1fxp5k_
  Resolved https://github.com/lvwerra/trl.git to commit fc468e0f3582de1aacd071fceb24265c619a8ef5
  Preparing metadata (setup.py) ... done
  Created wheel for trl: filename=trl-0.4.2.dev0-py3-none-any.whl size=54031 sha256=743598dd9af24d4f74dc845e873f71fb72efc6cb372a87f8e2acc065bcae2c41
  Stored in directory: /tmp/pip-ephem-wheel-cache-_vx_1o35/wheels/29/2a/0c/4cad07713d956ee7cbee0654570e3f3ccc30da3da04742db5d
Successfully built trl


In [7]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
tqdm.pandas()
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer
)
from trl import AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed
from trl.core import LengthSampler

In [8]:
def build_dataset(
    config, 
    dataset_name="allenai/real-toxicity-prompts", # SetFit/toxic_conversations ??
    input_min_text_length=5, 
    input_max_text_length=10
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    ds = load_dataset(dataset_name, split="train")

    def filter_fn(sample):
        toxicity = sample["prompt"]["toxicity"]
        return toxicity is not None and toxicity > 0.3

    ds = ds.filter(filter_fn, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        prompt = sample["prompt"]["text"]
        continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt + continuation)[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")

    ds = ds.train_test_split(test_size=0.2, shuffle=False)["train"]

    return ds

In [29]:
model_name="ybelkada/gpt-j-6b-sharded-bf16"

#model_name="google/flan-t5-base"
log_with=None
learning_rate=1.47e-5 * 2
mini_batch_size=1
gradient_accumulation_steps=1
num_steps=50
batch_size=256
num_epochs=1

In [30]:
config = PPOConfig(
    steps=num_steps,
    batch_size=batch_size,
    ppo_epochs=num_epochs,
    model_name=model_name,
    learning_rate=learning_rate,
    log_with=log_with,
    mini_batch_size=mini_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
)

# set seed before initializing value head for deterministic eval
set_seed(config.seed)

# We retrieve the dataloader by calling the `build_dataset` function.
min_input_length = 30
max_input_length = 40

dataset = build_dataset(config, input_min_text_length=min_input_length, input_max_text_length=max_input_length)

Using custom data configuration allenai--real-toxicity-prompts-eb8779dd2693db47
Found cached dataset json (/root/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Loading cached processed dataset at /root/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-77cd39dcb7404b1c.arrow


  0%|          | 0/35109 [00:00<?, ?ex/s]

In [31]:
# Now let's build the model, the reference model, and the tokenizer. We first load the model
# in bfloat16 to save memory using `transformers`.
model = AutoModelForCausalLM.from_pretrained(config.model_name, torch_dtype=torch.bfloat16)
# And then we pass the loaded model to `AutoModelForCausalLMWithValueHead`.
model = AutoModelForCausalLMWithValueHead.from_pretrained(model)

# T5
# model = AutoModelForSeq2SeqLM.from_pretrained(config.model_name, torch_dtype=torch.bfloat16)
# model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(model)

# We create a reference model by sharing 20 layers
ref_model = create_reference_model(model) #, num_shared_layers=20)

# We make sure to use `Adam` optimizer on the model parameters that require gradients.
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

# GPT-2 / GPT-J tokenizer has a pad token, but it is not eos_token by default. We need to set it to eos_token.
# only for this model.
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
    optimizer=optimizer,
)

In [28]:
# We then build the reward pipeline, we will use the toxicity model to compute the reward.
# We first load the toxicity model and tokenizer.
toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
# We load the toxicity model in fp16 to save memory.
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, torch_dtype=torch.float16).to(
    ppo_trainer.accelerator.device
)

# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}
output_min_length = 20
output_max_length = 30
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [22]:
model_save_path = "./tmp_models/{}-detoxified".format(model_name.replace('/', '-'))

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
        
    # Get response from model
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # Compute sentiment score # noqa
    texts = batch["response"]
    print(texts)
    toxicity_inputs = toxicity_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(
        ppo_trainer.accelerator.device
    )
    logits = toxicity_model(**toxicity_inputs).logits.float()
    toxicity_labels = (logits[:, 0]).tolist()

    rewards = [torch.tensor(output) for output in toxicity_labels]

    # Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

    # Save model every 100 epochs
    if epoch % 100 == 0:
        if ppo_trainer.accelerator.is_main_process:
            # ppo_trainer.save_pretrained(model_save_path) # this relies on huggingface_hub 
            ppo_trainer.accelerator.unwrap_model(ppo_trainer.model).save_pretrained(model_save_path)
            ppo_trainer.tokenizer.save_pretrained(model_save_path)


0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['The super PAC will use Facebook, Twitter and other digital platforms to reach women, Latinos, blacks and ', '<pad> To a child that would be a guardian of a parent.</s>', 'You’d have one two-fingered hand swinging from your face – a hand as sensitive', '<pad> chew</s>', '<pad> Did we use a bug fix with our iOS differences?</s>', '<pad> Gamosa</s>', 'Opiods unleash 60,000 dead, in several areas, across North America and Europe. The killings', '<pad> I understood possibly that the cases could now only be viewed in tandem with the Oscar nomination.</s>', "<pad> 'fatsi dad's desperate</s>", 'Andrew Humphreys’ resignation as a police officer is a deepignment by Humphrey', '<pad> My grandpa was right: you can’t fix dumb.</s>', 'There had been no American intervention by the rebel sides; the group, led by Jamia Ud-ul Sanader', '<pad> 18 November 1996</s>', '<pad> Angela Dawkins</s>', 'Swimmers need human excrement, but Olympic athletes will have to swim through it everyday if they', '<pad> H

0it [01:12, ?it/s]


ValueError: Responses are too short. Make sure they are at least 4 tokens long.

In [ ]:
#ppo_trainer.model.save()